In [ ]:
!pip install --q requests_html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 2.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import os
import requests
from requests_html import HTMLSession, AsyncHTMLSession
from bs4 import BeautifulSoup as bs
import asyncio
if asyncio.get_event_loop().is_running():
  import nest_asyncio
  nest_asyncio.apply()

In [ ]:
# Change this to specifiy the data URL and the  number of pages you want to scrape

session = AsyncHTMLSession()
headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15'}

page_rows = []
page_num = 1
res = 0
total = 1
headers_found = False

while res != total and page_num <= 5:
  url = f'http://senvol.com/5_material-results/?appSession=0LTD9N6NY50DFEC28SO9N86TP49WFI6R6GEO4242743X1UP61EW017ASO6LS75SQX9CX2NW0G6Z6G8S456523E2RT203784AYQ578JEI6N0Q1I1MF957ZES52XGY3K38&PageID=2&PrevPageID=&cpipage={page_num}&CPISortType=&CPIorderBy=&appSession=26D32R65NSJT4F758HXOR2LE407P1479FX808Z5288IX77J2J5XK0RJX1Q99KFV05V5FF5K5Z1PHFF754130M6P3SQZ7CU1KUE5J5O367I9OG0YOL8WD1HH6D2D3F802&PageID=2&PrevPageID=&cpipage=2&CPISortType=&CPIorderBy='
  page_num += 1

  r = await session.get(url,headers=headers)
  await r.html.arender(timeout=20, retries=3)
  resp = r.html.raw_html
  soup = bs(resp, 'html.parser')

  table = soup.find('table', {'id':re.compile(r'cbTable_')})
  rows = table.find_all('tr', {'id':re.compile(r'DataRow')})
  for row in rows:
    page_rows.append([item.text for item in row.find_all('td')])
    if not headers_found:
      header_raw = table.find('tr', class_=re.compile(r'cbResultSetTableHeader_'))
      header = [label.text for label in header_raw.find_all('th', class_=re.compile(r'cbResultSetHeaderCell'))]
      headers_found = True

  record_text = soup.find('td', class_=re.compile(r'cbResultSetRecordMessage_')).text
  nums = re.findall(r'\d+', record_text)
  res = nums[1]
  total = nums[2]

  print(f'page {page_num-1}: results {res} of {total} scraped')

senvol_df = pd.DataFrame(page_rows, columns=header)

[INFO] Starting Chromium download.
INFO:pyppeteer.chromium_downloader:Starting Chromium download.
100%|██████████| 183M/183M [00:01<00:00, 127Mb/s]
[INFO] Beginning extraction
INFO:pyppeteer.chromium_downloader:Beginning extraction
[INFO] Chromium extracted to: /root/.local/share/pyppeteer/local-chromium/1181205
INFO:pyppeteer.chromium_downloader:Chromium extracted to: /root/.local/share/pyppeteer/local-chromium/1181205


page 1: results 25 of 4236 scraped
page 2: results 25 of 4236 scraped
page 3: results 25 of 4236 scraped
page 4: results 25 of 4236 scraped
page 5: results 25 of 4236 scraped


In [ ]:
senvol_df.head()

In [ ]:
senvol_df.to_csv('./AM_MasterData.csv')